# Wrangler Hello World

Execute the following command on Wrangler to start a Spark cluster (Not due to an issue with the SAGA SLURM adaptor it must be executed from the Home directory):


    saga-hadoop --resource=slurm://localhost --queue=normal --walltime=59 --number_cores=24 --project=TG-MCB090174 --framework spark

In [ ]:
!saga-hadoop --resource=slurm://localhost \
             --queue=normal --walltime=59 --number_cores=24 \
             --project=TG-MCB090174 --framework spark

### Optional: investigate environment on Wrangler

In [ ]:
!squeue

In [ ]:
!java -version

In [ ]:
!/sbin/ifconfig

In [3]:
import pyspark

In [2]:
import os
os.environ["SPARK_LOCAL_IP"]="10.1.58.132"
%run ../util/init_spark_wrangler.py
from pilot_hadoop import PilotComputeService as PilotSparkComputeService


pilotcompute_description = {''
    "service_url": "spark://c251-132.wrangler.tacc.utexas.edu:7077",
    "spark.driver.host": os.environ["SPARK_LOCAL_IP"]
}

print "SPARK HOME: %s"%os.environ["SPARK_HOME"]
#print "PYTHONPATH: %s"%os.environ["PYTHONPATH"]

start = time.time()
pilot_spark = PilotSparkComputeService.create_pilot(pilotcompute_description=pilotcompute_description)
sc = pilot_spark.get_spark_context()
print "Spark Startup, %.2f"%(time.time()-start)

SPARK HOME: /home/01131/tg804093/work/spark-2.2.0-bin-hadoop2.6
Connect to Spark cluster: spark://c251-132.wrangler.tacc.utexas.edu:7077
spark.master=spark://c251-132.wrangler.tacc.utexas.edu:7077
spark.driver.host=10.1.58.132
spark.app.name=Pilot-Spark


OSError: [Errno 2] No such file or directory

In [5]:
rdd=sc.parallelize(range(10000), 4)

In [6]:
rdd.cache()

ParallelCollectionRDD[2] at parallelize at PythonRDD.scala:475

In [7]:
rdd.count()

10000

# Kafka

    saga-hadoop --resource=slurm://localhost    --queue=normal --walltime=180 --number_cores=32   --project=TG-MCB090174 --framework kafka

In [ ]:
!squeue -u tg804093

In [ ]:
from pykafka import KafkaClient
zkKafka='c251-118.wrangler.tacc.utexas.edu:2181'

client = KafkaClient(zookeeper_hosts=zkKafka)
#client = KafkaClient(hosts='c251-142.wrangler.tacc.utexas.edu:9092')
topic = client.topics['test']
producer = topic.get_sync_producer()
consumer = topic.get_simple_consumer()

In [ ]:
client.cluster.brokers

In [ ]:
producer.produce(str('Hello World dfasd dfads'))

In [ ]:
message = consumer.consume(block=False)
print("%d: %s"%(message.offset, message.value))

## Kafka and Spark Streaming

Kafka Integration only functional with spark-submit (see StreamingApp.py)

PySpark Init:

     /home/01131/tg804093/work/spark-2.0.0-bin-hadoop2.6/bin/pyspark --master spark://c251-121.wrangler.tacc.utexas.edu:7077 --packages  org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0

Download Spark Package:

    #!/bin/bash
    mvn dependency:get -Dartifact=org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0
    find $HOME/.m2/ -name "*.jar" |xargs cp -t /home/01131/tg804093/src/spark-streaming-kafka/

In [ ]:
files=!ls /home/01131/tg804093/src/spark-streaming-kafka/
jar_files = ",".join(["/home/01131/tg804093/" + i for i in files])

jar_files_colon=":".join(["/home/01131/tg804093/" + i for i in files])

In [ ]:
import os
#os.environ["PYSPARK_SUBMIT_ARGS"]="--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0"

%run util/init_spark_wrangler.py
from pilot_hadoop import PilotComputeService as PilotSparkComputeService

#os.environ["PYSPARK_SUBMIT_ARGS"]="--jars /home/01131/tg804093/src/spark-streaming-kafka/spark-streaming-kafka-0-10-assembly_2.11-2.0.0.jar"
#os.environ["SPARK_CLASSPATH"]=":".join(["/home/01131/tg804093/" + i for i in files])

#os.environ["SPARK_CLASSPATH"]="/home/01131/tg804093/spark-streaming-kafka-0-8_2.11-2.0.0.jar"


pilotcompute_description = {
    "service_url": "spark://c251-121.wrangler.tacc.utexas.edu:7077",
    #"spark.jars.packages": "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0",
    "spark.jars": "/home/01131/tg804093/spark-streaming-kafka-0-8_2.11-2.0.0.jar"
    #"spark.driver.host": os.environ["SPARK_LOCAL_IP"]
}

print "SPARK HOME: %s"%os.environ["SPARK_HOME"]
print "PYTHONPATH: %s"%os.environ["PYTHONPATH"]

start = time.time()
pilot_spark = PilotSparkComputeService.create_pilot(pilotcompute_description=pilotcompute_description)
sc = pilot_spark.get_spark_context()
print "Spark Startup, %.2f"%(time.time()-start)

In [ ]:
zkKafka='c251-118.wrangler.tacc.utexas.edu:2181'
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
ssc = StreamingContext(sc, 1)
kvs = KafkaUtils.createStream(ssc, zkKafka, "spark-streaming-consumer", {'test': 1})    

In [ ]:
lines = kvs.map(lambda x: x[1])
counts = lines.flatMap(lambda line: line.split(" ")) \
        .map(lambda word: (word, 1)) \
        .reduceByKey(lambda a, b: a+b)
counts.pprint()
ssc.start()
ssc.awaitTermination()